## 一些准备工作

In [6]:
# torch packets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np 
from sklearn import  metrics

# torch vision 
from torchvision import datasets, transforms

mnist_path = 'DATA'
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])


train_set = datasets.MNIST(root=mnist_path, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=mnist_path, train=False, transform=trans)
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

# 模型结构
class mnist_net(nn.Module):
    def __init__(self):
        super(mnist_net, self).__init__() # 为啥？
        self.fc1 = nn.Linear(28* 28 ,  500)
        self.fc2 = nn.Linear(500, 32)
        self.fc3 = nn.Linear(32, 10)
        pass
    
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu( self.fc1(x)  )
        x = F.relu( self.fc2(x)  )
        out = self.fc3(x)
        return out 
        pass 
    pass

the_model = mnist_net()
the_model.load_state_dict(torch.load('mnist_net'))

In [8]:
y_true = test_set.targets
y_pred2 = torch.empty(0, dtype= torch.long)
for batch_idx, (x, target) in enumerate(test_loader):
    y_pred_tmp = the_model.forward(x) 
    label_pred = torch.argmax( y_pred_tmp, dim= 1 )
    #print(label_pred)
    #print(target)
    y_pred2 = torch.cat( (y_pred2, label_pred)  ) # 连接两个向量
    pass

print('\nAccuracy Score of the Loaded Model: %g' %( metrics.accuracy_score(y_true, y_pred2)))


Accuracy Score of the Loaded Model: 0.9634


## Fast Gradient Sign Method

In [29]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, -0.5, 0.5) 
    # Return the perturbed image
    return perturbed_image

In [80]:
epsilon = 0.3

accuracy_list = []

for batch_idx, (data, target) in enumerate(test_loader):
    
    data.requires_grad = True
    
    #     print(target.size() ) # 128
    len_target = len(target)
    # target = torch.ones(len_target, dtype= torch.long)
    
    # 问题！！ 如何做 targeted attack ???? 
    
    # print(data.size() )  # torch.Size([128, 1, 28, 28]) 
    # 代表了 有 128 个样本
    
    output = the_model.forward(data)
    # loss = F.nll_loss(output, target)
    
    criterion = nn.CrossEntropyLoss()
    loss =   - criterion(output, target)
    # print(loss)
    
    init_pred = output.max(1, keepdim=True)[1]  # 干什么的？？？
    # print(init_pred[0])
    # print(output[0])
    
    the_model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    
    perturbed_data = fgsm_attack(data, epsilon, data_grad)
    
    y_adv = the_model.forward(perturbed_data)
    y_pred = the_model.forward(data)
    
    y_adv = torch.argmax(y_adv, dim= 1)
    y_pred = torch.argmax(y_pred, dim = 1)
    
    
    print(y_adv)
    print(y_pred)
    # print(target)
    # print(metrics.accuracy_score(y_pred, y_adv ))
    acc = metrics.accuracy_score(y_pred , y_adv )
    accuracy_list.append(acc)
    # print(acc) 
    # print(data_grad.size() ) # torch.Size([128, 1, 28, 28 ])  

    
    # print( data.max()  )
    # print( data.min() )
    # tensor(0.5000, grad_fn=<MaxBackward1>)
    # tensor(-0.5000, grad_fn=<MinBackward1>)
    
    
    break # only run once 
    pass

print('\nEpsilon:')
print(epsilon)

print('--'*12)

print('Accuracy Score:')
print(np.mean(accuracy_list))

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 2, 6, 9, 0, 5, 5, 9, 7, 3, 4, 9, 6, 6, 3,
        4, 0, 7, 6, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 6, 1, 9, 5, 4, 8, 9, 3, 7, 4, 6, 4, 3, 0, 2, 0,
        2, 9, 1, 7, 3, 2, 9, 8, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 6,
        1, 7, 6, 4, 6, 0, 4, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 2, 9, 2,
        8, 4, 7, 6, 7, 9, 0, 5])
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 9, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 1, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 4, 7, 6, 4, 9, 0, 5])

Epsilon:
0.3
------------------------
Accuracy Score:
0.8671875


MNIST 分类的时候，还是不要归一化成 【-0.5, 0.5 】这样

不习惯啊